In [1]:
import pandas as pd
import numpy as np
import main as sigmod
import similar_feature_grouping as similar_feature_grouping
import helper as helper

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gunesyurdakul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset_path="../datasets/2013_camera_specs/"

In [3]:
dataset_df = sigmod.create_brand_dataframe(dataset_path)
dataset_df = sigmod.compute_brand_blocking(dataset_df)
#pairs_df = sigmod.get_block_pairs_df(dataset_df)

  0%|          | 0/24 [00:00<?, ?it/s]

>>> Creating dataframe...



100%|██████████| 24/24 [00:02<00:00, 10.02it/s]
463it [00:00, 4623.20it/s]

>>> Dataframe created successfully!

>>> Computing blocking...


29787it [00:04, 6018.69it/s]


ended
>>> Blocking computed successfully!



### GROUPING SAME PRODUCTS FROM LABELLED SET

In [4]:
labelled_df = pd.read_csv('../datasets/sigmod_large_labelled_dataset.csv')

In [5]:
same_products = sigmod.grouping_same_products_from_labelled_set(labelled_df)

In [6]:
#They all represent the same products
same_products[4]

['www.ebay.com//55354',
 'www.ebay.com//53297',
 'www.pricedekho.com//1220',
 'www.ebay.com//43368',
 'www.ebay.com//47315',
 'www.ebay.com//45475',
 'www.ebay.com//55847',
 'www.ebay.com//43645',
 'www.ebay.com//44639',
 'www.ebay.com//48983',
 'www.ebay.com//46258',
 'www.ebay.com//47981',
 'www.ebay.com//59659',
 'www.ebay.com//44533',
 'www.ebay.com//60633',
 'www.ebay.com//48936',
 'www.ebay.com//45136',
 'www.ebay.com//56287',
 'www.ebay.com//44475',
 'www.ebay.com//46766',
 'www.ebay.com//58785',
 'www.ebay.com//46021',
 'www.ebay.com//52765',
 'www.ebay.com//43738',
 'www.ebay.com//46558',
 'www.gosale.com//1468',
 'www.ebay.com//58806',
 'www.ebay.com//44205',
 'www.ebay.com//46290',
 'www.ebay.com//59145',
 'www.ebay.com//57427',
 'www.ebay.com//54572',
 'www.ebay.com//56670',
 'www.ebay.com//47901',
 'www.ebay.com//56105',
 'www.ebay.com//46388',
 'www.ebay.com//55784',
 'www.ebay.com//56154',
 'www.ebay.com//45683',
 'www.ebay.com//43951',
 'www.ebay.com//48527',
 'www.ebay

### GET ALL KEYS and values

In [7]:
data_dict=sigmod.get_all_keys_values(dataset_path)
dataset_df['all_features'] = dataset_df['spec_id'].apply(lambda x: data_dict[x])

100%|██████████| 24/24 [00:02<00:00,  7.52it/s]


In [ ]:
dataset_df.head()

,source,spec_number,spec_id,page_title,brand,blocking_key,all_features
0,www.wexphotographic.com,154,www.wexphotographic.com//154,nikon coolpix aw120 digital camera - camouflag...,,[coolpix],"{'colour': 'Camouflage', 'movie mode': '1080p ..."
1,www.wexphotographic.com,553,www.wexphotographic.com//553,canon ixus 150 digital camera - red (9148b007a...,,[canon],"{'colour': 'Red', 'face detection technology':..."
2,www.wexphotographic.com,601,www.wexphotographic.com//601,fuji finepix s1 digital camera (p10nc12730a) -...,,[fujifilm],"{'colour': 'Black', 'face detection technology..."
3,www.wexphotographic.com,197,www.wexphotographic.com//197,nikon coolpix s5300 digital camera - black (vn...,,[coolpix],"{'colour': 'Black', 'face detection technology..."
4,www.wexphotographic.com,178,www.wexphotographic.com//178,fuji finepix s8600 digital camera - red (p10nc...,,[fujifilm],"{'colour': 'Red', 'face detection technology':..."


In [ ]:
all_text=similar_feature_grouping.prepare_text_for_word2vec(data_dict)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
emb_dim=200
model = Word2Vec(all_text, size=emb_dim, window=5, min_count=1, workers=4)
model.train(all_text,total_examples=model.corpus_count,epochs=model.epochs)

In [ ]:
model.most_similar('supply ability')

In [ ]:
model.save('word2vec_200_features')

### GROUPING FEATURES

In [ ]:
all_features={}
for features in data_dict.values():
    for feature in features.keys():
        if feature not in all_features.keys():
            all_features[feature]=0            
        all_features[feature]+=1

In [ ]:
all_features_filtered=[feature for feature,count in all_features.items() if count>200]

In [ ]:
len(all_features_filtered)

### CLUSTERING

In [ ]:
model= Word2Vec.load('word2vec_200_features')
words = all_features_filtered
wvs = model.wv[words]


In [ ]:
wvs.shape

### TSNE

In [ ]:
# visualize embeddings
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0, n_iter=1000, perplexity=20)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(wvs)
labels = words



In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

In [ ]:
plt.show()

In [ ]:
T.shape

#### KMeans

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk
%matplotlib inline

In [ ]:
NUM_CLUSTERS=50
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True,repeats=25)
assigned_clusters = kclusterer.cluster(T, assign_clusters=True)

In [ ]:
plt.figure(figsize=(30,20))
plt.scatter(T[:,0], T[:,1], c=assigned_clusters, cmap='hsv')
plt.scatter(np.array(kclusterer.means())[:,0], np.array(kclusterer.means())[:,1], marker='+', cmap='hsv')

for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

In [ ]:
dataset_df,cluster2features = similar_feature_grouping.get_grouped_features(dataset_df,assigned_clusters,labels)

In [ ]:
dataset_df = dataset_df.set_index('spec_id')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
dataset_df.loc[same_products[3]]

# FOR A SINGLE BLOCK - Nikon

In [ ]:
dataset_df.head(50)
blocking_key="polaroid"
dataset_df = dataset_df.explode('blocking_key')
grouped_df = dataset_df.groupby(['blocking_key'])
block_df = grouped_df.get_group(blocking_key)

In [ ]:
len(block_df)

In [ ]:
global counter
counter=0
block_df['page_title_sim']=""

In [ ]:
def get_sim_among_col(x,column_data):
    sim_array=np.zeros(len(column_data))
    i=0
    global counter
    for cell_data in column_data.values:
        sim_array[i]=helper.cosine_sim(helper.text_to_ngrams(x,3,'chars'),helper.text_to_ngrams(cell_data,3,'chars'))
        i+=1
    counter+=1
    if(counter%10==0):
        print(counter/len(column_data),np.mean(sim_array))
    return sim_array

In [ ]:
block_df['page_title']=block_df['page_title'].apply(lambda x: x.replace(blocking_key,''))
block_df['page_title_sim']=block_df['page_title'].apply(lambda x: get_sim_among_col(x,block_df['page_title']))

In [ ]:
block_df

In [ ]:
cluster2features

In [ ]:
block_df.iloc[np.argsort(-block_df['page_title_sim'].loc['buy.net//4236'])]

In [ ]:
block_df.iloc[0]['page_title']

In [ ]:
sigmod.extract_model_words(['hero 3','dfgh','we','16 mp'])

In [ ]:
helper.cosine_sim(n1,n2)

In [ ]:
for idx in range(NUM_CLUSTERS):
    dataset_df[idx]=dataset_df[idx].apply(lambda x: )

In [ ]:
'nikon sauask akska nikon sdsds'.replace('nikon','')